# Exercício: livros populares 

Dado um dataset com avaliações de livros, vamos encontrar os mais populares por dois critérios:

1. pela quantidade de avaliações

2. pela média da nota da avaliação

Motivação: construir um recomendador de livros populares.

## Baixando o dataset

Vamos usar o dataset `goodbooks-10k` criado para ser usado em problemas de recomendação. Ele contém cerca de 6 milhões de avaliações para os 10 mil livros mais populares.

Leia mais no [fast-ml](http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/).

Se tiver erro na execução da célula abaixo, baixe manualmente os arquivos do [github](https://github.com/zygmuntz/goodbooks-10k) e coloque os arquivos `books.csv` e `ratings.csv` em uma pasta chamada `data` neste diretório.

In [ ]:
# !mkdir -p data
# !wget -P data https://github.com/zygmuntz/goodbooks-10k/raw/master/books.csv
# !wget -P data https://github.com/zygmuntz/goodbooks-10k/raw/master/ratings.csv

In [ ]:
path = 'data' # se local
#path = 'dbfs:/FileStore/tables' # se usar notebook databricks

## Leitura dos dados

In [ ]:
books_df = spark.read.csv(f'{path}/books.csv', inferSchema=True)

In [ ]:
books_df.show(n=2)

### O que aconteceu aqui?

Parece que os nomes das colunas não foram lidos propriamente

In [ ]:
help(spark.read.csv)

### Corrigindo o problema

Se olharmos a documentação com calma, vamos ver esse pedaço:

```
    :param header: uses the first line as names of columns. If None is set, it uses the
                   default value, ``false``.
```

**Exercício:** Inclua esses parâmetro (para lermos a primeira linha como `header`) e efetue novamente a leitura

In [ ]:
books_df = ###

In [ ]:
books_df.show(n=2)

**Exercício:** Agora, faça a leitura do arquivo `data/ratings.csv`

In [ ]:
ratings_df = ###

In [ ]:
ratings_df.show(n=2)

É comum encontrar outros formatos de dados ao trabalhar com Spark. Saiba mais [aqui](https://eng.uber.com/hdfs-file-format-apache-spark/).

## Dica para os próximos exercícios

Para as próximas manipulações, é provável que usemos métodos do módulo `pyspark.sql.functions`, assim vamos importá-lo, juntamente com o módulo `pyspark.sql.types`.

A documentação do módulo `pyspark.sql` pode ser encontrada [neste endereço](https://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html).

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

### Inspecionando os tipos de cada coluna

É possível verificar o chamado `schema` do dataframe usando o método `printSchema`

In [ ]:
ratings_df.printSchema()

Note que, se não tivéssemos passado o parâmetro `inferSchema=True`, então, os tipos das colunas seriam diferentes:

In [ ]:
spark.read.csv(f'{path}/ratings.csv', header=True).printSchema()

Em nosso caso, o Spark conseguiu inferir corretamente os tipos de cada uma das colunas, mas caso ele não tivesse conseguido, poderíamos tentar forçar a conversão através do uso da função [cast](https://spark.apache.org/docs/2.4.4/api/python/_modules/pyspark/sql/column.html#Column.cast).

Por exemplo, para transformar a coluna `string` em inteiro, faríamos:

In [ ]:
spark.read.csv(f'{path}/ratings.csv', header=True).select('user_id', 'book_id', F.col('rating').cast('int')).printSchema()

## Parte 1: quantidade de avaliações de cada um dos livros

In [ ]:
count_reviews_df = ratings_df \
    .select('book_id', 'rating') \
    .groupBy('book_id') \
    .count()

In [ ]:
count_reviews_df.show(n=2)

### Maneiras alternativas para fazer o mesmo cálculo

```python
count_reviews_df = ratings_df \
    .groupBy('book_id') \
    .agg(F.count('rating').alias('count'))
```

Também é possível usar Spark SQL:
```python
ratings_df.createOrReplaceTempView('ratings')

count_reviews_query = """
    select book_id,
        count(rating) as count
    from ratings
    group by book_id
"""

count_reviews_df = spark.sql(count_reviews_query)
```

## Parte 2: média das notas de avaliações de cada um dos livros

**Exercício:** calcule essa média a partir do dataframe `ratings_df`

In [ ]:
avg_ratings_df = ###

In [ ]:
avg_ratings_df.show(n=2)

**Exercício:** faça o mesmo cálculo usando Spark SQL

In [ ]:
ratings_df.createOrReplaceTempView('ratings') # se você já rodou essa linha anteriormente, não é necessário rodá-la novamente

In [ ]:
avg_ratings_query = ###

In [ ]:
spark.sql(avg_ratings_query).show(n=2)

Note que `spark.sql(avg_ratings_query)` é um Spark DataFrame, assim como `avg_ratings_df`.

### Parte 3: junção dos dados de quantidade de avaliações e médias das notas das avaliações

Queremos um dataframe que contenha `book_id`, `count` e `mean_rating`.

In [ ]:
count_avg_ratings_df = count_reviews_df \
    .join(avg_ratings_df, on='book_id', how='inner')

In [ ]:
count_avg_ratings_df.show(n=2)

## Parte 4: junção dos dados de título e imagem

Gostaríamos agora de incluir no dataframe `count_avg_ratings_df` as colunas `title` e `image_url`.

**Exercício:** faça outra operação de `join`, desta vez utilizando os dataframes `count_avg_ratings_df` e `books_df` para incluir no dataframe as colunas desejadas.

In [ ]:
books_df.printSchema()

In [ ]:
count_avg_ratings_df = ###

In [ ]:
count_avg_ratings_df.show(n=2)

### Parte 5: visualização dos livros mais populares 

In [ ]:
from pyspark.sql.window import Window

In [ ]:
count_ordered_window = Window.orderBy(F.desc('count'))

In [ ]:
count_avg_ratings_df = count_avg_ratings_df \
    .withColumn('count_rank', F.row_number().over(count_ordered_window))

**Exercício:** da mesma forma, crie uma coluna chamada `avg_rank`, que calcula o rank segundo a coluna `mean_rating`

In [ ]:
avg_ordered_window = ###

In [ ]:
count_avg_ratings_df = ###

In [ ]:
count_avg_ratings_df.orderBy('count', ascending=False).show(n=2)

In [ ]:
count_avg_ratings_df.orderBy('mean_rating', ascending=False).show(n=2)

**Exercício:** formate a tabela de modo a ver os dez livros mais populares de acordo com cada método

Ao final, sua tabela deve ser como a abaixo:

|rank|according_to_count                                         |according_to_avg                                                 |
|----|-----------------------------------------------------------|-----------------------------------------------------------------|
|1   |The Hunger Games (The Hunger Games, #1)                    |The Complete Calvin and Hobbes                                   |
|2   |Harry Potter and the Sorcerer's Stone (Harry Potter, #1)   |ESV Study Bible                                                  |
|3   |To Kill a Mockingbird                                      |Attack of the Deranged Mutant Killer Monster Snow Goons          |

In [ ]:
top_10 = ###

In [ ]:
top_10.show(truncate=False)

### Extra: visualização das capas dos livros

Para essa seção, é necessário ter instalado a biblioteca [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html).

In [ ]:
from ipywidgets import widgets, Layout
from IPython.display import clear_output, HTML, Markdown, display
import requests

In [ ]:
top_10_imgs = count_avg_ratings_df.select(F.col('count_rank').alias('rank'), F.col('image_url').alias('according_to_count')) \
    .join(
        count_avg_ratings_df.select(F.col('avg_rank').alias('rank'), F.col('image_url').alias('according_to_avg')),
        on='rank') \
    .filter('rank <= 10')

In [ ]:
top_10_imgs.cache()

In [ ]:
top_10_imgs.show()

In [ ]:
def get_recommended_products(method, n):
    imgs = top_10_imgs.select('rank', F.col(f'according_to_{method}').alias('url')).limit(n).collect()
    return [(img.rank, img.url) for img in imgs]

def printmd(string):
    display(Markdown(string))

def make_horizontal_box(children): return widgets.HBox(children)

def make_vertical_box(children, width='auto', height='600px'):
    return widgets.VBox(children, layout=Layout(width=width, height=height))

def image_widget(url, layout=Layout(height='250px', width='150px', display='flex', align_items='center', border='solid white')):
    img_content = requests.get(url).content
    return widgets.Image(value=img_content, layout=layout)

def widgets_to_render(method, n):
    layout = Layout(height='250px', width='150px', display='flex', align_items='center', border='solid orchid')
    return [image_widget(elem[1]) if i > 0 else image_widget(elem[1], layout=layout) 
            for i, elem in enumerate(get_recommended_products(method, n))]

def print_on_button(string, color='lightblue'):
    button = widgets.Button(description=string, layout=Layout(width='300px'))
    button.style.button_color = color
    return button

def display_both_recommendations(n=3):
    boxes = [
        print_on_button('highest_count', color='lightblue'),
        make_horizontal_box(widgets_to_render('count', n))
    ]
    boxes += [print_on_button('highest_avg', color='lightpink'),
              make_horizontal_box(widgets_to_render('avg', n))]
    display(make_vertical_box(boxes))

In [ ]:
display_both_recommendations(n=5)